In [1]:
import scanpy as sc 
import metashells as ms
from metashells.simulation import create_simulated_adata

In [2]:
import anndata
import sys

print(f"Anndata version: {anndata.__version__}")
print(f"Python executable: {sys.executable}")

Anndata version: 0.12.6
Python executable: /ocean/projects/cis240075p/asachan/.conda/envs/metasheller-py311/bin/python


/tmp/ipykernel_8552/3463952342.py:4: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  print(f"Anndata version: {anndata.__version__}")


In [3]:
adata_with_labels_file = '/ocean/projects/cis240075p/asachan/datasets/TA_muscle/ERCC1_KO_mice/integrated_samples/scANVI_v2.h5ad'
adata = sc.read_h5ad(adata_with_labels_file)

/ocean/projects/cis240075p/asachan/.conda/envs/metasheller-py311/lib/python3.11/site-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
adata.X.min(), adata.X.max()

In [ ]:
adata.layers['raw_count'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

adata.layers['normalized_counts'] = adata.X
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata.X.min(), adata.X.max()

In [ ]:
sc.tl.pca(adata)
sc.pl.pca(adata, color='sample')

In [ ]:
import os 

results_dir = './tutorial_outputs'
os.makedirs(results_dir, exist_ok=True)

In [ ]:
from metashells.seashells import SeaShells 

# to parallelize, simply submit multiple jobs running the following code

conch = SeaShells(
    adata_full = adata, 
    results_dir = results_dir,
    cells_per_metacell = 75, # recommended is 75 but can go as low as ~20 if really needed
    sample_col = 'sample'
)
conch.run()